In [ ]:
# Step 0: Install required packages
!pip install requests transformers

In [ ]:
%pip install -q -U "google-genai>=1.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 16.3 MB/s eta 0:00:00


In [ ]:
# Install requests for API calls
!pip install requests


In [ ]:
import os
import json
import requests
from google import genai

# Load API keys from Colab secrets
# GOOGLE_API_KEY is already set in environment variables
client = genai.Client()  # automatically uses GOOGLE_API_KEY from os.environ
MODEL_ID = "gemini-2.5-flash-preview-05-20"
SERPER_API_KEY = "64d6ae585c82b13dbf6085e4ffbbeb499fb4db4d"

In [ ]:
# -----------------------------
# Step 1: Gemini API dream interpretation
# -----------------------------
def interpret_dream_gemini(user_query):
    """
    Sends the dream query to Gemini 2.5 Flash model and returns interpretation.
    """
    try:
        response = client.models.generate_content(
            model=MODEL_ID,
            contents=f"You are an expert dream interpreter. Explain the meaning of this dream in a shorter way:\n{user_query}"
        )
        return response.text
    except Exception as e:
        return f"Error: {e}"


In [ ]:
def get_web_insights_serper(user_query, top_k=3):
    """
    Uses Serper API to get top-k related snippets for the dream query.
    """
    try:
        url = "https://google.serper.dev/search"
        headers = {"X-API-KEY": SERPER_API_KEY, "Content-Type": "application/json"}
        payload = {"q": f"{user_query} dream meaning"}
        response = requests.post(url, headers=headers, json=payload)
        data = response.json()

        # Try multiple possible keys for results
        results = data.get("organic", []) or data.get("organic_results", []) or data.get("items", [])
        snippets = []
        for item in results[:top_k]:
            # Some items may have "snippet" or "description"
            snippet = item.get("snippet") or item.get("description")
            if snippet:
                snippets.append(snippet)

        if not snippets:
            snippets.append("No web insights found or API returned empty results.")

        return snippets

    except Exception as e:
        return [f"Error fetching web insights: {e}"]


In [ ]:
# -----------------------------
# Step 3: Merge interpretations
# -----------------------------
def merge_dream_interpretation(user_query):
    """
    Generates structured JSON output combining Gemini LLM interpretation
    and top web insights. make the ultimate interpretation short use only matched point view in a organize way.
    do not make the interpretation too long i want short and precise interpretation.
    """
    llm_response = interpret_dream_gemini(user_query)
    web_snippets = get_web_insights_serper(user_query)

    ultimate_interpretation = f"{llm_response}"
    if web_snippets:
        ultimate_interpretation += " Merged with key themes from web insights and make it short: " + "; ".join(web_snippets)

    json_output = {
        "dream_query": user_query,
        "ai_interpretation": llm_response,
        "web_insights": web_snippets,
        "ultimate_interpretation": ultimate_interpretation
    }

    try:
        print("JSON Output:\n", json.dumps(json_output, indent=2))
    except Exception as e:
        print(f"JSON Encode Error: {e}")

    return json_output

In [ ]:
# -----------------------------
# Step 4: Example run
# -----------------------------
user_query = "I dreamt of flying over the ocean"
dream_result = merge_dream_interpretation(user_query)

JSON Output:
 {
  "dream_query": "I dreamt of flying over the ocean",
  "ai_interpretation": "This dream signifies a powerful sense of freedom and mastery over your emotions and inner world, offering a new perspective.",
  "web_insights": [
    "Dreams about flying over calm, shallow ocean waters often symbolize freedom and emotional clarity. Straight, crossing paths may represent life ...",
    "I call this an anxiety dream. To float above the earth speaks to a desire to be above the worries that you're currently dealing with. To die in ...",
    "When water appears in dreams, emotions, feelings, and spirituality are often associated with it. Flying over water may represent cleansing, purity, or even life ..."
  ],
  "ultimate_interpretation": "This dream signifies a powerful sense of freedom and mastery over your emotions and inner world, offering a new perspective. Merged with key themes from web insights and make it short: Dreams about flying over calm, shallow ocean waters often s